In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import shapely
import pytest
import tkinter as tk

In [ ]:
# Define custom colour palette

humannature = {'GC Dark Green': '#294238',
               'GC Light Green': '#b2d235',
               'GC Mid Green': '#50b748',
               'GC Orange': '#f57821',
               'GC Light Grey': '#e6ebe3'
               }

In [ ]:
# SQL server details 
server = 'dblistener1' #Details omitted for security reasons
database = 'information_centre' #Details omitted for security reasons
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = '''
""" with engine.connect() as connection: allvisits = pd.read_sql(query, connection) connection.close()